### 코사인 유사도를 사용한 영화추천 코드 실습

In [1]:
# 데이터는 여기에서
# https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset?resource=download
# 파일 중 movies_metadata.csv 데이터 사용

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [ ]:
data = pd.read_csv('movies_metadata.csv',low_memory=False)
data = pd.DataFrame(data)
data.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [ ]:
# 해당 데이터에서 영화 제목에 해당하는 title열과 줄거리에 해당하는 overview만 사용
# 좋아하는 영화를 입력하면, 해당 영화의 줄거리와 유사한 줄거리의 영화를 찾아 추천하는 시스템

In [ ]:
# 데이터가 커서 상위 2만개의 데이터만 쓰고 싶다면
# data = data.head(20000)

In [ ]:
# overview 열에 존재하는 모든 null값 카운트 하여 출력
print(data['overview'].isnull().sum())

954


In [ ]:
# 위에 처럼 null값이 존재할 경우 
# pandas의 dropna() 결측값 행을 모두 제거
# pandas의 fillna() 특정값으로 채워넣기
# 예제에서는 null값을 그냥 공백으로 채워줌
# data['overview'] = data['overview'].fillna('')

# 나는 그냥 제거해서 해보자
data = data.dropna(subset=['overview'])
print(data['overview'].isnull().sum())

0


In [ ]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['overview'])

# 기존 예제보다 데이터가 2만개 정도 더 많음
print('TF-IDF 행렬의 크기(shape) :',tfidf_matrix.shape)

TF-IDF 행렬의 크기(shape) : (44512, 75827)


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print('코사인 유사도 연산 결과 :',cosine_sim.shape)

코사인 유사도 연산 결과 : (44512, 44512)


In [ ]:
title_to_index = dict(zip(data['title'], data.index))

# 영화 제목 Father of the Bride Part II의 인덱스를 리턴
idx = title_to_index['Father of the Bride Part II']
print(idx)

4


In [ ]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당 영화의 인덱스를 받아온다.
    idx = title_to_index[title]

    # 해당 영화와 모든 영화와의 유사도를 가져온다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬한다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아온다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 얻는다.
    movie_indices = [idx[0] for idx in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴한다.
    return data['title'].iloc[movie_indices]

In [ ]:
# 결과적으로... 잘안나옴 :(, fillna로 공백을 유지해서 다시 도전
get_recommendations('The Dark Knight Rises')

31143             Deadly Daycare
19286            The One Percent
44918                  Once More
45106    Nicostratos the Pelican
33008       White Cannibal Queen
15225             Father and Son
44912    Three Songs About Lenin
4132          The Luzhin Defence
34321            Maine Pyar Kiya
9844             Come and Get It
Name: title, dtype: object

In [ ]:
data = pd.read_csv('movies_metadata.csv',low_memory=False)
data = data.head(20000)
data['overview'] = data['overview'].fillna('')